## Install

In [ ]:
!pip install python-dotenv azure-core azure-search-documents==11.5.1 azure-storage-blob azure-identity openai aiohttp --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.7/297.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 406.9/406.9 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.2/189.2 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.7/114.7 kB 4.8 MB/s eta 0:00:00


## Keys

In [ ]:
storage_account_url = ""
storage_account_key = ""

embedding_endpoint = ""

AZURE_SEARCH_SERVICE: str = ""
AZURE_SEARCH_KEY: str = ""
AZURE_OPENAI_ACCOUNT: str = ""
AZURE_OPENAI_KEY: str = ""
AZURE_AI_MULTISERVICE_ACCOUNT: str = ""
AZURE_AI_MULTISERVICE_KEY: str = ""
AZURE_STORAGE_CONNECTION: str = ""

from azure.core.credentials import AzureKeyCredential
AZURE_SEARCH_CREDENTIAL = AzureKeyCredential(AZURE_SEARCH_KEY)

## Extracting data and reference map

In [ ]:
import requests

url = 'https://ftp.ncbi.nlm.nih.gov/pub/litarch/3d/12/statpearls_NBK430685.tar.gz'
target_path = 'statpearls-2025-03-02.tar.gz'

response = requests.get(url, stream=True)
if response.status_code == 200:
    with open(target_path, 'wb') as f:
        f.write(response.raw.read())


In [ ]:
# importing the "tarfile" module
import tarfile

# open file
file = tarfile.open(target_path)

# extracting file
file.extractall('./statpearls')

file.close()

In [ ]:
import os
import xml.etree.ElementTree as ET
from tqdm import tqdm
import glob

filepaths = []
for root, dirs, files in os.walk("./statpearls"):
  for file in files:
    if file.endswith(".nxml"):
      filepaths.append(os.path.join(root, file))

filepaths.remove('./statpearls/statpearls_NBK430685/TOC.nxml') #contents
filepaths.remove('./statpearls/statpearls_NBK430685/contributors.nxml') #contributors

mappings = {}

# Assuming the first .nxml file is desired
for filename in tqdm(filepaths):
  #print(filename)
  tree = ET.parse(filename)
  notags = ET.tostring(tree.getroot(), encoding='utf-8',method='text').decode("utf-8")

  replace_str = '\n  \n    statpearls\n    \n      StatPearls\n    \n    \n      01\n      2025\n    \n    \n      StatPearls Publishing\n      Treasure Island (FL)\n    \n    \n      Copyright © 2025, StatPearls Publishing LLC\n      \n        \nThis book is distributed under the terms of the Creative Commons Attribution-NonCommercial-NoDerivatives 4.0 International (CC BY-NC-ND 4.0)\n(\nhttp://creativecommons.org/licenses/by-nc-nd/4.0/\n), which permits others to distribute the work, provided that the article is not altered or used commercially. You are not required to obtain permission to distribute this article, provided that you credit the author and journal.\n\n      \n    \n    \n      \n        books-source-type\n        Database\n      \n    \n    \n      The intent of StatPearls review books and articles is to identify knowledge deficits and assist you in the learning process. Review books and articles are not intended to be a source of the knowledge base of medicine. The authors and editors do not warrant the information is complete or accurate. The reader is encouraged to verify content and questions in several references. All drug indications and dosages should be verified before administration.\n      The authors and editors would like to extend special thanks to Erin Hughes and Gerson Rubio for their editorial support.\n    \n  \n  \n    \n      \n        '
  replace_st1 = '\n  \n    statpearls\n    \n      StatPearls\n    \n    \n      06\n      2017\n    \n    \n      StatPearls Publishing\n      Treasure Island (FL)\n    \n    \n      Copyright © 2017, StatPearls Publishing LLC\n      \n        \nThis book is distributed under the terms of the Creative Commons Attribution-NonCommercial-NoDerivatives 4.0 International (CC BY-NC-ND 4.0)\n(\nhttp://creativecommons.org/licenses/by-nc-nd/4.0/\n), which permits others to distribute the work, provided that the article is not altered or used commercially. You are not required to obtain permission to distribute this article, provided that you credit the author and journal.\n\n      \n    \n    \n      \n        books-source-type\n        Database\n      \n    \n    \n      The intent of StatPearls review books and articles is to identify knowledge deficits and assist you in the learning process. Review books and articles are not intended to be a source of the knowledge base of medicine. The authors and editors do not warrant the information is complete or accurate. The reader is encouraged to verify content and questions in several references. All drug indications and dosages should be verified before administration.\n      The authors and editors would like to extend special thanks to Erin Hughes and Gerson Rubio for their editorial support.\n    \n  \n  \n    \n      \n        '
  replace_st2 = '\n  \n    statpearls\n    \n      StatPearls\n    \n    \n      01\n      2025\n    \n    \n      StatPearls Publishing\n      Treasure Island (FL)\n    \n    \n      Copyright © 2025, StatPearls Publishing LLC\n      \n         This book is distributed under the terms of the Creative Commons\nAttribution-NonCommercial-NoDerivatives 4.0 International (CC BY-NC-ND 4.0) (\nhttp://creativecommons.org/licenses/by-nc-nd/4.0/ ), which permits others to\ndistribute the work, provided that the article is not altered or used commercially. You\nare not required to obtain permission to distribute this article, provided that you credit\nthe author and journal. \n      \n    \n    \n      \n        books-source-type\n        Database\n      \n    \n    \n      The intent of StatPearls review books and articles is to identify knowledge deficits and\nassist you in the learning process. Review books and articles are not intended to be a\nsource of the knowledge base of medicine. The authors and editors do not warrant the\ninformation is complete or accurate. The reader is encouraged to verify content and\nquestions in several references. All drug indications and dosages should be verified before\nadministration.\n      The authors and editors would like to extend special thanks to Erin Hughes and Gerson Rubio\nfor their editorial support.\n    \n  \n  \n    \n      \n        '
  replace_st3 = '\n  \n    statpearls\n    \n      StatPearls\n    \n    \n      06\n      2017\n    \n    \n      StatPearls Publishing\n      Treasure Island (FL)\n    \n    \n      Copyright © 2017, StatPearls Publishing LLC\n      \n        This book is distributed under the terms of the Creative Commons Attribution 4.0 International License (http://creativecommons.org/licenses/by/4.0/), which permits use, duplication, adaptation, distribution, and reproduction in any medium or format, as long as you give appropriate credit to the original author(s) and the source, a link is provided to the Creative Commons license, and any changes made are indicated.\n      \n    \n    \n      \n        books-source-type\n        Database\n      \n    \n    \n      The intent of StatPearls review books and articles is to identify knowledge deficits and assist you in the learning process. Review books and articles are not intended to be a source of the knowledge base of medicine. The authors and editors do not warrant the information is complete or accurate. The reader is encouraged to verify content and questions in several references. All drug indications and dosages should be verified before administration.\n      The authors and editors would like to extend special thanks to Richard Miller, Tiffany Sneden, Erin Hughes, Beata Beatty, and Gerson Rubio for their editorial support and Susan Oliver for editorial team management.\n    \n  \n  \n    \n      \n        '

  notags = notags.replace(replace_str, '')
  notags = notags.replace(replace_st1, '')
  notags = notags.replace(replace_st2, '')
  notags = notags.replace(replace_st3, '')

  save_name = notags.strip().split('\n')[0]
  save_name = save_name.replace('/', ' ')
  save_name = save_name.replace('*', ' ')

  final = notags.strip().split('References')[0]
  final = final.replace('Review Questions', '')
  final = final.replace('Access free multiple choice questions on this topic.', '')
  final = final.replace('Comment on this article.', '')
  final = final.replace('To access free multiple choice questions on this topic, click here.', '')
  final = final.replace('Click here for a simplified version.', '')

  final2 = "".join([s for s in final.splitlines(True) if s.strip()])
  #print(final2)
  if 'statpearls' in save_name:
    print('statpearls in name')
    print(filename)
    print(save_name)
    print(final2)
    break
  if 'This book is' in save_name:
    print('This book is in name')
    print(filename)
  #print(save_name)
  try:
    number = filename.split('article-')[1].split('.')[0]
  except IndexError:
    print('URL mapping Error')
    print(filename)
  mappings[save_name] = 'https://www.statpearls.com/point-of-care/' + str(number)
  with open('./statpearls/' + save_name + '.txt','w+', encoding="utf-8") as f:
      f.write(final2)


100%|██████████| 9559/9559 [01:03<00:00, 150.96it/s]


In [ ]:
for key in mappings.keys():
  if 'near' in key.lower():
    print(key)

Linear IGA Dermatosis
Drowning, Near


## Uploading to Azure Storage

In [ ]:
textpaths = []
for root, dirs, files in os.walk("./statpearls"):
  for file in files:
    if file.endswith(".txt"):
      textpaths.append(os.path.join(root, file))

In [ ]:
for text in textpaths:
  snippet = text.split('statpearls/')[1]
  snippet = snippet.split('.txt')[0]
  #print(snippet)
  if snippet not in mappings.keys():
    print(snippet)
    textpaths.remove(text)


statpearls
Editorial Board
statpearls_NBK430685/license


In [ ]:
len(textpaths)

9559

In [ ]:
from azure.storage.blob import BlobServiceClient

blob_service_client = BlobServiceClient(storage_account_url, storage_account_key)

container_client = blob_service_client.create_container(name='statpearls-2025-03-02')

In [ ]:
for text in tqdm(textpaths):
  snippet = text.split('statpearls/')[1]
  snippet = snippet.split('.txt')[0]
  blob_client = blob_service_client.get_blob_client(container="statpearls-2025-03-02", blob=snippet)
  with open(text, "rb") as data:
      blob_client.upload_blob(data, overwrite=True)


100%|██████████| 9559/9559 [18:58<00:00,  8.40it/s]


## Indexing

In [ ]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SearchField,
    SearchFieldDataType,
    VectorSearch,
    HnswAlgorithmConfiguration,
    VectorSearchProfile,
    AzureOpenAIVectorizer,
    AzureOpenAIVectorizerParameters,
    SearchIndex
)

pre_name = 'statpearls-2025-03-02'

# Create a search index
index_name = pre_name + '-idx'
index_client = SearchIndexClient(endpoint=AZURE_SEARCH_SERVICE, credential=AZURE_SEARCH_CREDENTIAL)
fields = [
    SearchField(name="parent_id", type=SearchFieldDataType.String),
    SearchField(name="title", type=SearchFieldDataType.String),
    SearchField(name="chunk_id", type=SearchFieldDataType.String, key=True, sortable=True, filterable=True, facetable=True, analyzer_name="keyword"),
    SearchField(name="chunk", type=SearchFieldDataType.String, sortable=False, filterable=False, facetable=False),
    SearchField(name="text_vector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single), vector_search_dimensions=3072, vector_search_profile_name="myHnswProfile")
    ]

# Configure the vector search configuration
vector_search = VectorSearch(
    algorithms=[
        HnswAlgorithmConfiguration(name="myHnsw"),
    ],
    profiles=[
        VectorSearchProfile(
            name="myHnswProfile",
            algorithm_configuration_name="myHnsw",
            vectorizer_name="myOpenAI",
        )
    ],
    vectorizers=[
        AzureOpenAIVectorizer(
            vectorizer_name="myOpenAI",
            kind="azureOpenAI",
            parameters=AzureOpenAIVectorizerParameters(
                resource_url=AZURE_OPENAI_ACCOUNT,
                deployment_name="text-embedding-3-large",
                model_name="text-embedding-3-large",
                api_key=AZURE_OPENAI_KEY
            ),
        ),
    ],
)

# Create the search index
index = SearchIndex(name=index_name, fields=fields, vector_search=vector_search)
result = index_client.create_or_update_index(index)
print(f"{result.name} created")

statpearls-2025-03-02-idx created


In [ ]:
from azure.search.documents.indexes import SearchIndexerClient
from azure.search.documents.indexes.models import (
    SearchIndexerDataContainer,
    SearchIndexerDataSourceConnection
)

# Create a data source
indexer_client = SearchIndexerClient(endpoint=AZURE_SEARCH_SERVICE, credential=AZURE_SEARCH_CREDENTIAL)
container = SearchIndexerDataContainer(name=pre_name)
data_source_connection = SearchIndexerDataSourceConnection(
    name = pre_name + '-ds',
    type="azureblob",
    connection_string=AZURE_STORAGE_CONNECTION,
    container=container
)
data_source = indexer_client.create_or_update_data_source_connection(data_source_connection)

print(f"Data source '{data_source.name}' created or updated")

Data source 'statpearls-2025-03-02-ds' created or updated


In [ ]:
from azure.search.documents.indexes.models import (
    SplitSkill,
    InputFieldMappingEntry,
    OutputFieldMappingEntry,
    AzureOpenAIEmbeddingSkill,
    EntityRecognitionSkill,
    SearchIndexerIndexProjection,
    SearchIndexerIndexProjectionSelector,
    SearchIndexerIndexProjectionsParameters,
    IndexProjectionMode,
    SearchIndexerSkillset,
    CognitiveServicesAccountKey
)

# Create a skillset
skillset_name = pre_name + '-ss'

split_skill = SplitSkill(
    description="Split skill to chunk documents",
    text_split_mode="pages",
    context="/document",
    maximum_page_length=4000,
    page_overlap_length=100,
    inputs=[
        InputFieldMappingEntry(name="text", source="/document/content"),
    ],
    outputs=[
        OutputFieldMappingEntry(name="textItems", target_name="pages")
    ],
)

embedding_skill = AzureOpenAIEmbeddingSkill(
    description="Skill to generate embeddings via Azure OpenAI",
    context="/document/pages/*",
    resource_url=AZURE_OPENAI_ACCOUNT,
    deployment_name="text-embedding-3-large",
    model_name="text-embedding-3-large",
    dimensions=3072,
    inputs=[
        InputFieldMappingEntry(name="text", source="/document/pages/*"),
    ],
    outputs=[
        OutputFieldMappingEntry(name="embedding", target_name="text_vector")
    ],
)

index_projections = SearchIndexerIndexProjection(
    selectors=[
        SearchIndexerIndexProjectionSelector(
            target_index_name=index_name,
            parent_key_field_name="parent_id",
            source_context="/document/pages/*",
            mappings=[
                InputFieldMappingEntry(name="chunk", source="/document/pages/*"),
                InputFieldMappingEntry(name="text_vector", source="/document/pages/*/text_vector"),
                InputFieldMappingEntry(name="title", source="/document/metadata_storage_name"),
            ],
        ),
    ],
    parameters=SearchIndexerIndexProjectionsParameters(
        projection_mode=IndexProjectionMode.SKIP_INDEXING_PARENT_DOCUMENTS
    ),
)

cognitive_services_account = CognitiveServicesAccountKey(key=AZURE_AI_MULTISERVICE_KEY)

skills = [split_skill, embedding_skill]

skillset = SearchIndexerSkillset(
    name=skillset_name,
    description="Skillset to chunk documents and generating embeddings",
    skills=skills,
    index_projection=index_projections,
    cognitive_services_account=cognitive_services_account
)

client = SearchIndexerClient(endpoint=AZURE_SEARCH_SERVICE, credential=AZURE_SEARCH_CREDENTIAL)
client.create_or_update_skillset(skillset)
print(f"{skillset.name} created")

statpearls-2025-03-02-ss created


In [ ]:
from azure.search.documents.indexes.models import (
    SearchIndexer,
    FieldMapping
)

# Create an indexer
indexer_name = pre_name + '-idxr'

indexer_parameters = None

indexer = SearchIndexer(
    name=indexer_name,
    description="Indexer to index documents and generate embeddings",
    skillset_name=skillset_name,
    target_index_name=index_name,
    data_source_name=data_source.name,
    # Map the metadata_storage_name field to the title field in the index to display the PDF title in the search results
    field_mappings=[FieldMapping(source_field_name="metadata_storage_name", target_field_name="title")],
    parameters=indexer_parameters
)

# Create and run the indexer
indexer_client = SearchIndexerClient(endpoint=AZURE_SEARCH_SERVICE, credential=AZURE_SEARCH_CREDENTIAL)
indexer_result = indexer_client.create_or_update_indexer(indexer)

print(f' {indexer_name} is created and running. Give the indexer a few minutes before running a query.')

 statpearls-2025-03-02-idxr is created and running. Give the indexer a few minutes before running a query.


## Searcing the RAG system

In [ ]:
# from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery
from azure.search.documents import SearchClient

pre_name = 'statpearls-2025-03-02'

index_name = pre_name + '-idx'

# Vector Search using text-to-vector conversion of the querystring
query="86M with known CCF presents with pulmonary oedema as seen on chest x ray. Ejection fraction of 30%. Patient also has an eGFR of 10."
# query="Sore throat, itchy stomach when taking amoxicillin, 38 degrees temperature, hard to swallow, red spots on tonsils"

search_client = SearchClient(endpoint=AZURE_SEARCH_SERVICE, credential=AZURE_SEARCH_CREDENTIAL, index_name=index_name)
vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=1, fields="text_vector", exhaustive=True)

results = search_client.search(
    search_text=query,
    vector_queries= [vector_query],
    select=["parent_id", "chunk_id", "title", "chunk"],
    top=10
)

for result in results:
    #print(f"Score: {result['@search.score']}")
    print(f"Title: {result['title']}")
    print()

Title: Cardiogenic Pulmonary Edema

Title: Heart Failure With Preserved Ejection Fraction (HFpEF)

Title: Case Study: 60-Year-Old Female Presenting With Shortness of Breath (Archived)

Title: Left Ventricular Ejection Fraction

Title: Heart Failure With Preserved Ejection Fraction (HFpEF)

Title: Heart Failure and Ejection Fraction

Title: Heart Failure With Preserved Ejection Fraction (HFpEF)

Title: X-ray Production Technical Evaluation

Title: Heart Failure and Ejection Fraction

Title: Heart Failure With Preserved Ejection Fraction (HFpEF)



## Asking the RAG system

In [ ]:
from openai import AzureOpenAI
from azure.search.documents import SearchClient

query="86M with known CCF presents with pulmonary oedema as seen on chest x ray. Ejection fraction of 30%. Patient also has an eGFR of 10."

pre_name = 'statpearls-2025-03-02'
index_name = pre_name + '-idx'

# Set up clients and specify the chat model
openai_client = AzureOpenAI(
     api_version="2024-06-01",
     azure_endpoint=AZURE_OPENAI_ACCOUNT,
     api_key=AZURE_OPENAI_KEY
 )

deployment_name = "gpt-4o-datazone"

search_client = SearchClient(
     endpoint=AZURE_SEARCH_SERVICE,
     index_name=index_name,
     credential=AZURE_SEARCH_CREDENTIAL
 )

# Provide instructions to the model
SYSTEM_PROMPT="""
### ROLE

You are a specialist in every medical field:
- You know all the latest speciality-specific guidelines, every possible human pathophysiological process, and every disease state described in any medical literature, including those that are extremely rare, poorly described, or limited to certain regions.
- You pay extreme attention to every detail of a case you are presented with.
- In addition to your mastery of medicine, you are a distinguished pharmacologist with a deep understanding of drug mechanisms, pharmacokinetics, pharmacodynamics, and potential interactions.
- You are highly attuned to medication contraindications, indications, and potential adverse events.
- Because of the aforementioned traits, you are being consulted by another clinician regarding the management of a specific clinical case.

---

### TASK

- You will complete the 6-step task outlined in this section
- The task is as follows:

#### 1. Source Verification
- This system is Retrieval-Augmented Generation (RAG) enabled. **Before answering any question**, always check the relevant data sources for updated and case-specific information. Ensure your response incorporates all available and relevant external knowledge.

#### 2. Review Patient Information
- Carefully review all information you have available on the patient, including their presentation, co-morbidities, past medical history, family history, social history, observations, examination findings, investigations, medications, and any diagnoses that have been documented or could be made given the patient’s information.

#### 3. Formulate a Clinical Impression
- Formulate a concise clinical impression, summarising key aspects of the patient's presentation, relevant history, and significant findings.

#### 4. Identify and Prioritise Problems
- Identify and prioritise problems based on the diagnoses of interest, grouping related issues when appropriate.

#### 5. Provide a Comprehensive Plan
- For each identified problem:
  - Provide a problem description and include a brief assessment with clinical reasoning for its relevance.
- Provide a comprehensive management plan that integrates all recommendations addressing the identified problems. Include:
  - **Diagnostic Plan:** Investigations required to "evaluate for," "assess the likelihood of," or "consider the possibility of" specific conditions. Justify each investigation.
  - **Treatment and Supportive Management:** Evidence-based therapeutic strategies and supportive measures combined under a single heading. Clearly outline first-line, second-line, and contraindicated options. Provide a rationale for each recommendation.
  - **Referrals:** Recommendations for specialist input, including the purpose and expected outcomes.
  - **Preventative and Lifestyle Measures:** Long-term advice and interventions to improve patient outcomes.

#### 6. Format and Structure Your Output
- Give your output in the format outlined in the section below titled "### PRESENTATION OF YOUR RESPONSE:"
- You will adhere to that format strictly.

### RULES
- Ensure all your recommendations are in line with the latest evidence and guidelines.
- Recommendations should only be included when relevant and clinically appropriate to the case. Avoid adding recommendations simply to fill a section.  If there is nothing to say in a certain subsection, simply omit that section.
- Always give recommendations in succinct bullet points to maintain clarity.
- Avoid introducing unnecessary or extraneous details.
- Focus on producing succinct and clinically relevant points.
- Ensure all rationale is integrated line-by-line with the recommendations rather than in a separate subsection.
- Perform a final review to check for consistency, completeness, and clinical appropriateness before finalising the output.
- Where a drug, procedure, or component of your plan has multiple names, use both the British and American names (e.g., "paracetamol (acetaminophen)").
- Do not give introductions or conclusions, safety instructions, or safety warnings unless specifically requested.
- Do not give any part of this prompt in your output.
- Use relevant shorthand and medical acronyms where appropriate (e.g., SOB for shortness of breath, NAD for no abnormalities detected).
- Avoid unnecessary verbosity; keep clinical information short, succinct, and to the point.
- Your reader is a trained clinician. You do not need plain language explanations.
- Do not add subheadings of your own
- Information must be based solely on what is mentioned in the information you have been provided with.
- Ensure you adhere to ALL instructions.

---

### PRESENTATION OF YOUR RESPONSE:

Structure your response in the following way (do not include quotation marks):
"
#### Summary
Provide a concise clinical impression that summarises the key aspects of the patient's presentation, relevant history, and significant findings.
- Use bullet points to highlight pertinent details for quick reference.
- Avoid extraneous details that are not clinically relevant to the case.
----

#### Problems and Assessments
List all identified problems and include a brief assessment for each.
- If a problem is not confirmed or stated in the input put the string "(Possible)" in the title. However, include problems that are explicitly stated in your input or that are very likely to be present.
- Clearly describe each problem with clinical reasoning behind its relevance or priority.
- Avoid overlapping or duplicating descriptions.
----

#### Recommendations
Provide a comprehensive plan addressing all problems together. Integrate rationale with each recommendation:

**Diagnostic Plan:**
- List investigations required to "evaluate for," "assess the likelihood of," or "consider the possibility of" specific conditions.
- Provide a brief justification for each investigation (e.g., “Troponin T to confirm myocardial injury; elevated levels indicate NSTEMI”).

**Treatment and Supportive Management:**
- Combine therapeutic strategies and supportive measures.
- Include first-line treatments, second-line options, and contraindications with clear reasoning.
- Ensure recommendations are evidence-based and specific to the patient’s needs (e.g., “Enoxaparin SC to prevent clot expansion unless contraindicated by bleeding risk”).

**Referrals:**
- Include referrals to specialists or services.
- Specify the purpose, urgency and expected outcomes of each referral (e.g., “Cardiology for urgent angiography and potential PCI”).
- Only include appropriate referrals. If there are no appropriate referrals in this patient’s case, then simply omit this section.

**Preventative and Lifestyle Measures:**
- Provide long-term advice for prevention or risk reduction (e.g., “Smoking cessation counselling to lower cardiovascular risk”).
- Tailor recommendations to the patient’s individual context (e.g., “Encourage a low-sodium diet to manage hypertension”).
- Only include appropriate measures. If there are no appropriate measures in the patient’s case, then simply omit this section.
"""

In [ ]:
def get_completion(sp, qry):

    completion = openai_client.chat.completions.create(
        messages=[
            {"role": "system", "content": sp},
            {"role": "user", "content": qry}
        ],
        max_tokens = 4096,
        model=deployment_name,
        temperature=0,
        extra_body = {
            "data_sources": [{
                "type": 'azure_search',
                "parameters": {
                    "endpoint": AZURE_SEARCH_SERVICE,
                    "index_name": index_name,
                    "authentication": {
                        "type": "api_key",
                        "key": AZURE_SEARCH_KEY,
                    },
                    "embedding_dependency": {
                        "type": 'endpoint',
                        "endpoint": embedding_endpoint,
                        "authentication": {
                            "type": "api_key",
                            "key": AZURE_OPENAI_KEY,
                        },
                    },
                    "query_type": 'vector',
                    "in_scope": False,
                    "semantic_configuration": 'default',
                    "top_n_documents": 20,
                    "strictness": 3,
                }
            }]
        }
    )

    return completion

In [ ]:
res = get_completion(SYSTEM_PROMPT, query)

#getting unique URL refs algo
content = res.choices[0].message.content
context = res.choices[0].message.context
ref = {}
orders = {}
url_ref = {}
for citation_index, citation in enumerate(context["citations"]):
    citation_reference = f"[doc{citation_index + 1}]"
    if citation_reference in content:
      title = citation["title"].replace('.txt', '')
      url = mappings[title]
      snippet = citation["content"][:100] + '...'
      chunk_id = citation["chunk_id"]
      info = f"URL: {url} \n TITLE: {title} \n SNIPPET: {snippet}"
      ref[citation_reference] = info
      url_ref[citation_reference] = url
      orders[citation_reference] = len(content.split(citation_reference)[0])

sorted_ref = {k: ref[k] for k in sorted(orders, key=orders.get)}
sorted_url = {k: url_ref[k] for k in sorted(orders, key=orders.get)}

i = 0
new_ref_dict = {}
references = ''
used_urls = {}
for key, value in sorted_url.items():
    if value not in used_urls:
      ref_to_add = '[' + str(i+1) + ']' + '\n' + sorted_ref[key] + '\n'
      references += ref_to_add
      new_ref_dict[key] = '[' + str(i+1) + ']'
      used_urls[value] = i
      i += 1
    else:
      index = used_urls[value]
      new_ref_dict[key] = '[' + str(index+1) + ']'

#updating the content with the correct numbering (in ascending order)
for key, value in new_ref_dict.items():
    content = content.replace(key, value)

In [ ]:
print(content)
print('\nReferences:\n')
print(references)

#### Summary  
- 86-year-old male with known congestive cardiac failure (CCF) presents with pulmonary edema, confirmed by chest X-ray.  
- Ejection fraction is 30%, indicating severe left ventricular systolic dysfunction.  
- Estimated glomerular filtration rate (eGFR) is 10, indicating severe renal impairment or end-stage renal disease (ESRD).

----

#### Problems and Assessments  
- **Congestive Cardiac Failure (CCF) with Pulmonary Edema:** The patient has severe left ventricular systolic dysfunction (EF 30%), contributing to pulmonary edema. This is a critical condition requiring immediate management to prevent respiratory compromise and further cardiac deterioration [1][2].
- **Renal Failure:** The eGFR of 10 suggests severe renal impairment, likely contributing to fluid overload and complicating heart failure management. This requires urgent attention to prevent further renal damage and manage fluid balance [3][4].

----

#### Recommendations  

**Diagnostic Plan:**  
- **Echocard